## Homework 02 – Vector Search

### Q1. Embedding the query

In [1]:
!pip install fastembed

In [2]:
import numpy as np
from fastembed import TextEmbedding

# Create the embedding model
model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# Embed the query
query = "I just discovered the course. Can I join now?"
# R: model.embed([query]) returns a generator that yields embeddings for each input string (here, just one string in a list)
q = next(model.embed([query])) # R: next(...) gets the first (and only) embedding from the generator

# Check the shape and minimal value of the embedding
print("Shape of the embedding:", q.shape)
print("Minimal value in the embedding:", np.min(q))

Shape of the embedding: (512,)
Minimal value in the embedding: -0.11726373885183883


### Q2. Cosine similarity with another vector


In [3]:
# R: This computes the dot product of the embedding with itself, which should be equal to the square of its norm
print(q.dot(q)) 
# R: This computes the Euclidean norm (length) of the embedding vector
print(np.linalg.norm(q)) 
# R: This will display the embedding vector for the query
q

1.0000000000000002
1.0


array([-7.63946260e-02, -7.30555505e-02,  5.86501763e-02,  3.92670571e-02,
       -1.41797115e-02, -4.68342854e-02,  2.69862822e-02,  3.49039060e-02,
        1.53419217e-03, -1.10407363e-02,  5.57556285e-02, -2.88360142e-02,
       -4.87239498e-02, -9.90923923e-02,  6.02579764e-02, -1.86105379e-02,
       -8.81040139e-03, -3.01994831e-02, -1.42541815e-02, -3.87884212e-02,
       -2.76125662e-02, -6.94957533e-03,  3.20653319e-02, -5.17838394e-03,
        8.37959894e-02, -8.87329606e-02, -7.30260625e-02,  5.92846802e-02,
        4.07805674e-02,  7.72684822e-02, -4.60029697e-02,  3.98426895e-02,
        1.74821510e-02,  8.70263712e-03, -3.09291054e-02,  2.20470391e-02,
        4.76479896e-02,  1.76566196e-02, -3.62013627e-02, -3.53408062e-02,
       -5.92537995e-03,  2.97091202e-02,  8.05883752e-02,  1.07132722e-02,
       -5.38192519e-02, -9.62881111e-03, -1.17263739e-01,  3.36546374e-02,
        7.05763168e-03,  7.70159791e-03, -5.17482529e-02,  5.46955574e-02,
       -7.80289904e-02,  

In [4]:
doc = "Can I still join the course after the start date?"
# Get the embedding for the document
doc_vector = next(model.embed([doc]))
# Compute cosine similarity (dot product) between the query vector (q) and document vector (doc) 
cos_simil = q.dot(doc_vector)
# Because both vectors are normalized (length 1), the dot product is equal to their cosine similarity
print("Cosine similarity:", cos_simil)

Cosine similarity: 0.9008528895674548


### Q3. Ranking by cosine

In [5]:
documents = [
    {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
     'section': 'General course-related questions',
     'question': 'Course - Can I still join the course after the start date?',
     'course': 'data-engineering-zoomcamp'},
    {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
     'section': 'General course-related questions',
     'question': 'Course - Can I follow the course after it finishes?',
     'course': 'data-engineering-zoomcamp'},
    {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
     'section': 'General course-related questions',
     'question': 'Course - When will the course start?',
     'course': 'data-engineering-zoomcamp'},
    {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
     'section': 'General course-related questions',
     'question': 'Course - What can I do before the course starts?',
     'course': 'data-engineering-zoomcamp'},
    {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
     'section': 'General course-related questions',
     'question': 'How can we contribute to the course?',
     'course': 'data-engineering-zoomcamp'}
]

# Embed the text field for each document
text_embeddings = list(model.embed(doc['text'] for doc in documents))

# Stack embeddings into a matrix
V = np.stack(text_embeddings)

# Compute cosine similarity for each document with the query vector q
cosines = V.dot(q)

# Print similarities and the index of the highest one
print("Cosine similarities:", cosines)
print("Document index with the highest similarity:", np.argmax(cosines))

Cosine similarities: [0.76296847 0.81823782 0.80853974 0.7133079  0.73044992]
Document index with the highest similarity: 1


### Q4. Ranking by cosine, version two
Is it different from Q3? If yes, why?
Yes, it’s different from Q3 because concatenating question and text gives more context, which can improve the match.

In [ ]:
# Concatenate question and text for each document
full_texts = [doc['question'] + ' ' + doc['text'] for doc in documents]

# Embed the concatenated fields
full_embeddings = list(model.embed(full_texts))  

# Stack embeddings into a matrix
V_full = np.stack(full_embeddings)

# Compute cosine similarity for each document with the query vector q
cosines_full = V_full.dot(q)

# Print similarities and the index of the highest one
print("Cosine similarities (full_text):", cosines_full)
print("Document index with the highest similarity (full_text):", np.argmax(cosines_full))

Cosine similarities (full_text): [0.85145432 0.84365942 0.8408287  0.7755158  0.80860078]
Document index with the highest similarity (full_text): 0


### Q5. Selecting the embedding model


In [7]:
smallest_model = TextEmbedding(model_name="BAAI/bge-small-en")
smallest_dim = next(smallest_model.embed([query]))
print("Shape of the smallest embedding:", smallest_dim.shape)

Shape of the smallest embedding: (384,)


### Q6. Indexing with qdrant (2 points)


In [10]:
import requests
from qdrant_client import QdrantClient, models

# Download and filter documents
docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

# Use the small embedding model handle
model_handle = "BAAI/bge-small-en"
EMBEDDING_DIMENSIONALITY = 384
collection_name = "ml_zoomcamp_fqa"

# Connect to Qdrant
client = QdrantClient("http://localhost:6333")

# Recreate collection to avoid errors if rerunning
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

# Create points using models.Document (Qdrant will embed)
points = []
for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    point = models.PointStruct(
        id=i,
        vector=models.Document(text=text, model=model_handle),
        payload=doc
    )
    points.append(point)

# Upsert points
client.upsert(
    collection_name=collection_name,
    points=points
)

# Search with the Q1 query, using models.Document for embedding
query = "I just discovered the course. Can I join now?"
results = client.query_points(
    collection_name=collection_name,
    query=models.Document(text=query, model=model_handle),
    limit=1,
    with_payload=True
)

print("Highest score in the results:", results.points[0].score)

/tmp/ipykernel_86430/2548308992.py:29: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


Highest score in the results: 0.8703172
